## 1. Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

Для этого мы проведём следующие работы:
1. Ознакомимление и первичный анализ даных
2. Предобработка: удаление пропусков, нормализация, категорирование.
3. Анализ, вычисление целевых метрик
4. Формирование выводов

### Шаг 1. Откройте файл с данными и изучите общую информацию. 

In [1]:
# импортируем пандас и читаем данные
import pandas as pd
data = pd.read_csv('/datasets/data.csv')

In [2]:
# посмотрим что за данные
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [3]:
data.head(15)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


### Вывод

### Шаг 2. Предобработка данных

### Обработка пропусков

In [4]:
#смотрим количество пропусков столбца 'days_employed' и что это за данные

#смотрим что это за данные
data[data['days_employed'].isnull()]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,среднее,1,гражданский брак,1,M,пенсионер,0,NaN,сыграть свадьбу
26,0,NaN,41,среднее,1,женат / замужем,0,M,госслужащий,0,NaN,образование
29,0,NaN,63,среднее,1,Не женат / не замужем,4,F,пенсионер,0,NaN,строительство жилой недвижимости
41,0,NaN,50,среднее,1,женат / замужем,0,F,госслужащий,0,NaN,сделка с подержанным автомобилем
55,0,NaN,54,среднее,1,гражданский брак,1,F,пенсионер,1,NaN,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...,...,...
21489,2,NaN,47,Среднее,1,женат / замужем,0,M,компаньон,0,NaN,сделка с автомобилем
21495,1,NaN,50,среднее,1,гражданский брак,1,F,сотрудник,0,NaN,свадьба
21497,0,NaN,48,ВЫСШЕЕ,0,женат / замужем,0,F,компаньон,0,NaN,строительство недвижимости
21502,1,NaN,42,среднее,1,женат / замужем,0,F,сотрудник,0,NaN,строительство жилой недвижимости


Скорее всего, пропуски связаны с ошибками при создании таблицы, т.к. какой-то закономерности (вроде отсуствия стажа в принципе у таких кредиторов) не просматривается.

Так же можно заметить, что столбец "total_income" у таких сотрудников тоже NaN. Проверим  пустые данные по этому столбцу:


In [5]:
data[data['total_income'].isnull()]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,среднее,1,гражданский брак,1,M,пенсионер,0,NaN,сыграть свадьбу
26,0,NaN,41,среднее,1,женат / замужем,0,M,госслужащий,0,NaN,образование
29,0,NaN,63,среднее,1,Не женат / не замужем,4,F,пенсионер,0,NaN,строительство жилой недвижимости
41,0,NaN,50,среднее,1,женат / замужем,0,F,госслужащий,0,NaN,сделка с подержанным автомобилем
55,0,NaN,54,среднее,1,гражданский брак,1,F,пенсионер,1,NaN,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...,...,...
21489,2,NaN,47,Среднее,1,женат / замужем,0,M,компаньон,0,NaN,сделка с автомобилем
21495,1,NaN,50,среднее,1,гражданский брак,1,F,сотрудник,0,NaN,свадьба
21497,0,NaN,48,ВЫСШЕЕ,0,женат / замужем,0,F,компаньон,0,NaN,строительство недвижимости
21502,1,NaN,42,среднее,1,женат / замужем,0,F,сотрудник,0,NaN,строительство жилой недвижимости


То же количество строк. 
Можно предположить, что пустые значения у тех людей, по которым нет данных о работе - стаже и доходе. 
Возможно эти данные не прогрузились из-за ошибки. Но может это и нормально.

Так как данные с пропусками составляют почти 10% от общих, видится неправильным их дропать. 
Чтобы не потерять имеющиеся данные, заполним пропуски в этих двух столбцах на медианные значения. 
Это не должно сильно повлиять на результат и выводы.

In [6]:
#найдём медианные значения по столбцам "days_employed" и "total_income"
days_median = data['days_employed'].median()
income_median = data['total_income'].median()

#заменим пустые ячейки на медианные значения
data['total_income'] = data['total_income'].fillna(value=income_median)
data['days_employed'] = data['days_employed'].fillna(value=days_median)

#и проверим как теперь с пропусками
data.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

## Вывод
Избавились от пропущенных значений, заменив их на медианные. Не очень красиво, но на итоговые результаты задания повлиять не должно.

# Замена типа данных

In [7]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Видим, что с вещественными типами у нас всё те же два столбца - "days_employed" и "total_income".

Разберемся с первым.
Очевидно, что 'days_employed' должна содержать целочисленные неотрицательные значения. Но данные там другие, включая отрицательные и очень большие числа как для дней(если этот банк на планете Земля). При выгрузке были допущены ошибки. 

Поэтому мы приведём данные в этом столбце к целочисленным значениям, а отрицательные заменим на положительные, для красоты (идентифицировать пенсионеров мы можем по другому столбцу.) 

Положительные значения только у людей с типом занятости "пенсионер" и "безработный". Кроме того, числа слишком большие для рабочих дней. Можно предположить, что данные выгружались из разных систем, и для пенсионеров данные брались не в днях, а часах.
Попробуем их привести к общему формату и значениям.

С безработными сложнее. Для них значения в этом столбце сложно распознать, т.к. даже в днях стаж получаетя слишком большой.
Но т.к. данные у нас по этому столбцу не используются - мы их просто не будем трогать.

In [8]:
#сменим тип данных столбца на целое число и возьмём по модулю
data['days_employed'] = data['days_employed'].astype('int')
data['days_employed'] = data['days_employed'].abs()

data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null int64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null float64
purpose             21525 non-null object
dtypes: float64(1), int64(6), object(5)
memory usage: 2.0+ MB


In [9]:
# Нормируем значения в столбце "days_employed" для пенсионеров (в дни, а не часы). 
# Для этого напишем функцию, которая меняет значение для пенсионеров. 

def hours_to_days(row):
    income = row['income_type']
    days = row['days_employed']
    if income in 'пенсионер':
        days_new = days/24
    else:
        days_new = days
    return days_new


# применяем функцию и заменяем значения в столбце
data['days_employed'] = data.apply(hours_to_days, axis=1)

# ещё разок изменим тип, т.к. после функции он поплыл на float (кстати, почему?)
data['days_employed'] = data['days_employed'].astype('int')

data.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,5623,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,14177,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,926,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,2879,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,152,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,6929,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,2188,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [10]:
# Кайф. Теперь сделаем целочисленным total_income. Вероятнее всего там сумма в рублях, но для наших задач копейками можно пренебречь.

data['total_income'] = data['total_income'].astype('int')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null int64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null int64
purpose             21525 non-null object
dtypes: int64(7), object(5)
memory usage: 2.0+ MB


## Вывод
Теперь у нас только целочисленные и строковые значения.
Мы избавились от float, а также причесали значения данных в столбце "days_employed".

### Обработка дубликатов

In [11]:
# поищем полные дубликаты
data.duplicated().sum()

54

Теперь более детально посмотрим на каждый строковый столбец и поищем дубликаты, которые помешают дальнейшей обработке.

In [12]:
# смотрим на значения в столбце "education"
data['education'].value_counts()

среднее                13750
высшее                  4718
СРЕДНЕЕ                  772
Среднее                  711
неоконченное высшее      668
ВЫСШЕЕ                   274
Высшее                   268
начальное                250
Неоконченное высшее       47
НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
НАЧАЛЬНОЕ                 17
Начальное                 15
ученая степень             4
Ученая степень             1
УЧЕНАЯ СТЕПЕНЬ             1
Name: education, dtype: int64

In [13]:
# приводим к нижнему регистру и проверяем
data['education'] = data['education'].str.lower()
data['education'].value_counts()

среднее                15233
высшее                  5260
неоконченное высшее      744
начальное                282
ученая степень             6
Name: education, dtype: int64

Ручной поиск по другим столбцам не показал каких-то проблем с дубликатами. Кроме столбца "purpose", но с ним отдельная песня.

In [14]:
#ещё раз глянем на дубликаты и удалим их
#data.duplicated().sum()
data.drop_duplicates(inplace=True)

### Вывод

Помимо полных дубликатов, которые вероятно появились в следствии ошибок при выгрузке, были найдены и "скрытые дубликаты", связанные с регистром в строках. Ручная обработка регистра позволила выявить большее количество полных дубликатов строк, что положительно сказалось на качестве данных.

### Лемматизация

In [15]:
# смотрим на значения столбца
data['purpose'].value_counts()

свадьба                                   791
на проведение свадьбы                     768
сыграть свадьбу                           765
операции с недвижимостью                  675
покупка коммерческой недвижимости         661
операции с жильем                         652
покупка жилья для сдачи                   651
операции с коммерческой недвижимостью     650
покупка жилья                             646
жилье                                     646
покупка жилья для семьи                   638
строительство собственной недвижимости    635
недвижимость                              633
операции со своей недвижимостью           627
строительство жилой недвижимости          624
покупка недвижимости                      621
покупка своего жилья                      620
строительство недвижимости                619
ремонт жилью                              607
покупка жилой недвижимости                606
на покупку своего автомобиля              505
заняться высшим образованием      

Судя по всему, можно выделить следующие цели: Операции с жильём и недвижимостью (собственная и коммерческая), операции с автомобилями, кредиты на образование и свадьба.
Попробуем лемматизировать данные из столбца и проверить теорию.

In [16]:
# загружаем лемматизатор
from pymystem3 import Mystem
m = Mystem()

# создадим список лемм из всех значений столбца
purposes_lemm = []
purposes = data['purpose'].unique()
for row in purposes:
    purposes_lemm += m.lemmatize(row)

# немного причешем и посмотрим, какие из них уникальные:
unique_lemm = []
for each in purposes_lemm:
    if each not in unique_lemm:
        unique_lemm.append(each)
        
print(unique_lemm)

['покупка', ' ', 'жилье', '\n', 'приобретение', 'автомобиль', 'дополнительный', 'образование', 'сыграть', 'свадьба', 'операция', 'с', 'на', 'проведение', 'для', 'семья', 'недвижимость', 'коммерческий', 'жилой', 'строительство', 'собственный', 'подержать', 'свой', 'со', 'заниматься', 'сделка', 'подержанный', 'получение', 'высокий', 'профильный', 'сдача', 'ремонт']


### Вывод

В целом понятно. Действительно, цели кредитов можно разбить на 4 большие категории:
кредиты на недвижимость
кредиты на автомобили
кредиты на образование
кредиты на свадьбу

### Категоризация данных

Для начала категоризируем самое сложное - цели кредита.
Как мы выяснили, их можно разбить на три категории - недвижимость, автомобили, образование, свадьба.
Для отнесения к той или иной категории будем лемматизировать цели и искать в них ключевые слова для каждой категории.

In [17]:
# напишем функцию для поиска лемм и отнесения строки к 4м категориям
def find_lemmas(row):
    # определяем категории
    category_1 = ['жилье', 'недвижимость']
    category_2 = ['автомобиль']
    category_3 = ['образование']
    category_4 = ['свадьба']
    
    # лемматизируем и возвращаем ID категории
    lemmas = m.lemmatize(row)
    for each in lemmas:
        if each in category_1:
            return 1
        if each in category_2:
            return 2
        if each in category_3:
            return 3
        if each in category_4:
            return 4


# применяем функцию к столбцу целей, а результат запишем в отдельный столбец
data['purpose_id'] = data['purpose'].apply(find_lemmas)
# т.к. я писал без try-except (потому что я вручную проверил, но можно было и с ними), проверим не появилось ли в столбце пустых значений
data['purpose_id'].unique()


array([1, 2, 3, 4])

In [18]:
# ну и сразу создадим датафрейм со словарём
p_data = {'purpose':['жильё', 'автомобиль', 'образование', 'свадьба'], 'purpose_id':[1,2,3,4]}
purpose_dict = pd.DataFrame(p_data)
purpose_dict

,purpose,purpose_id
0,жильё,1
1,автомобиль,2
2,образование,3
3,свадьба,4


Продолжаем. 
Теперь изучим что у нас с образованием. Для нас уже выделил поле с ID, проверим его.

In [19]:
# забацаем словарь
education_dict = data[['education', 'education_id']]
# сбросим дубли и проверим
education_dict = education_dict.drop_duplicates().reset_index(drop=True)
education_dict

,education,education_id
0,высшее,0
1,среднее,1
2,неоконченное высшее,2
3,начальное,3
4,ученая степень,4


Не лучший вариант словаря, но да ладно, для наших целей подойдёт.

Теперь посмотрим на семейный статус. Там тоже предусмотрительно добавлен ID, так что поступаем так же как с образованием.

In [20]:
family_status_dict = data[['family_status', 'family_status_id']]
family_status_dict = family_status_dict.drop_duplicates().reset_index(drop=True)
family_status_dict

,family_status,family_status_id
0,женат / замужем,0
1,гражданский брак,1
2,вдовец / вдова,2
3,в разводе,3
4,Не женат / не замужем,4


Отлично! 
Попробуем категоризировать доходы. Похоже, что данные нам предоставлены в рублях. Поэтому категорировать будем исходя из реальной жизнив в России:
Если доход < 50000 - категория "Низкий доход"
50000 < доход < 150000 - 'Средний доход'
150000 < доход < 250000 - 'Высокий доход'
Доход > 250000 - 'Очень высокий доход'

In [21]:
# напишем функцию для категоризации доходов
def income_cats(row):
    income = row['total_income']
    if income < 50000:
        return 1
    if 50000 <= income < 150000:
        return 2
    if 150000 <= income < 250000:
        return 3
    if income >= 250000:
        return 4
    
income_dict_data = {'income_id':[1,2,3,4], 'income_cat':['Низкий доход', 'Средний доход','Высокий доход','Очень высокий доход']}
income_dict = pd.DataFrame(income_dict_data)
income_dict

,income_id,income_cat
0,1,Низкий доход
1,2,Средний доход
2,3,Высокий доход
3,4,Очень высокий доход


In [22]:
# применим функцию и проверим нет ли упущенных значений
data['income_id'] = data.apply(income_cats, axis=1)
data['income_id'].isna().sum()

0

Теперь посмотрим на столбец 'children' и его данные.
Видно, что данные в этом столбце следующие: [ 1,  0,  3,  2, -1,  4, 20,  5]
Очевидно, что значения 20 и -1 являются ошибочными. 20 детей конечно возможно, но не у 70 человек!
Так как строк с этими ошибками не много, просто удалим их.

In [23]:
#data['children'].unique()
#data[data['children'] == 20].count()

#удаляем строки со значениями
data = data[data['children'] != -1]
data = data[data['children'] != 20]

#проверяем
#data['children'].unique()

#ресет индексов
data = data.reset_index(drop=True)

Итак, мы категорировали то, что имело смысл категорировать и получили новые таблицы-словари.
Теперь разобьём нашу изначальную таблицу так, чтобы с ней было удобно работать с учётом проведённой категоризации.

In [24]:
# формируем новый датафрейм, с которым и будем проводить дальнейшие рассчёты
credit_data = data[['children', 'education_id', 'family_status_id', 'debt', 'income_id', 'purpose_id']]
print(credit_data.head(10))

   children  education_id  family_status_id  debt  income_id  purpose_id
0         1             0                 0     0          4           1
1         1             1                 0     0          2           2
2         0             1                 0     0          2           1
3         3             1                 0     0          4           3
4         0             1                 1     0          3           4
5         0             0                 1     0          4           1
6         0             0                 0     0          3           1
7         0             1                 0     0          2           3
8         2             0                 1     0          2           4
9         0             1                 0     0          2           1


Теперь таблица не очень читаема, зато отлично поддаётся анализу:)

### Вывод

Мы провели категоризацию данных по образованию, семейному статусу и целям получения кредита. Это позволило упростить итоговую таблицу, вынеся данные в словари.
После мы сформировали датафрейм с данным, которые нам необходимы для дальнейшего анализа.

### Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [25]:
# напишем функцию, которая на вход берёт название столбца, а на выходе даёт датафрейм с группированными значениями этого 
# столбца и процентом должников по данной группировке

def count_debts(column):     
    # подсчет по столбцу debt - это кол-во кредитов, сумма - количество должников
    result = credit_data.groupby(column).agg(total=('debt','count'), have_debt=('debt','sum'))
    # переводим в проценты
    result['percent_of_debtors'] = ((result['have_debt'] / result['total'])*100)
    result['percent_of_debtors'] = result['percent_of_debtors'].round(1).astype(str)+'%'
    return result


In [26]:
# проверим нашу функцию на детях))))
children_result = count_debts('children').reset_index()

# сфоромируем итоговую таблицу, отсортируем
children_final = children_result[['children','percent_of_debtors']]
children_final = children_final.sort_values(by='percent_of_debtors', ascending=False).reset_index(drop=True)

children_final

,children,percent_of_debtors
0,4,9.8%
1,2,9.5%
2,1,9.2%
3,3,8.2%
4,0,7.5%
5,5,0.0%


### Вывод

Можно видеть, что заёмщики с детьми отдают кредиты вовремя реже, чем заёмщики без детей. Вероятно, расходы на детей  являются более приоритетными, чем возврат кредита.
Интересным является и то, что все заёмщики с 5ю детьми вообще не имеют задолженностей. Причин может быть несколько: 5 детей делают тебя очень ответственным, за 5 детей кредиты гасятся по льготной или социальной программе, либо у нас недочёт в данных и выгруза долго по этим заёмщикам не верна.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [27]:
# применим нашу функцию к столбцу с ID семейного статуса
family_status_result = count_debts('family_status_id')

# для наглядности добавим данные из словаря 
family_status_result = family_status_result.merge(family_status_dict, on='family_status_id', how='left')

# сфоромируем итоговую таблицу, отсортируем
family_status_final = family_status_result[['family_status','percent_of_debtors']]
family_status_final = family_status_final.sort_values(by='percent_of_debtors', ascending=False).reset_index(drop=True)


family_status_final

,family_status,percent_of_debtors
0,Не женат / не замужем,9.8%
1,гражданский брак,9.3%
2,женат / замужем,7.6%
3,в разводе,7.1%
4,вдовец / вдова,6.6%


### Вывод

Интересные данные. Самыми должниками являются две категории - не женатые и живущие гражданским браком (тоже не женатые).
Можно предположить, что это связано с отсутствием как юридической, так и психологической ответственности. Ведь кредиты, взятые супругами считаются общей ответственностью законодательно. Кроме того, выплачивать кредит если оба супруга имеют доход очевидно проще.
На будущее, было бы интересно поисследовать взаимосвязь этого параметра с другими - с количеством детей и доходами. 

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [28]:
# применим нашу чудо-функцию
income_result = count_debts('income_id')

# добавим данные по категориям из словаря
income_result = income_result.merge(income_dict, on='income_id', how='left')

# сформируем итоговую таблицу
income_total = income_result[['income_cat','percent_of_debtors']]
income_total = income_total.sort_values(by='percent_of_debtors', ascending=False).reset_index(drop=True)
income_total

,income_cat,percent_of_debtors
0,Средний доход,8.3%
1,Высокий доход,8.3%
2,Очень высокий доход,6.9%
3,Низкий доход,6.2%


### Вывод

Интересно, что процент должников ниже среди людей с очень низким и очень высоким доходом. Бедные становятся беднее, богатые богаче. В принципе, ничего нового))

- Как разные цели кредита влияют на его возврат в срок?

In [29]:
# чудо-функция
purpose_result = count_debts('purpose_id')

# добавим словарь
purpose_result = purpose_result.merge(purpose_dict, on='purpose_id', how='left')

# сформируем итоговую таблицу
purpose_total = purpose_result[['purpose','percent_of_debtors']]
purpose_total = purpose_total.sort_values(by='percent_of_debtors', ascending=False).reset_index(drop=True)
purpose_total

,purpose,percent_of_debtors
0,автомобиль,9.3%
1,образование,9.3%
2,свадьба,7.9%
3,жильё,7.3%


### Вывод

Авто и образование - вещи в жизни небольшие, можно и не отдавать. А вот свадьба и жильё - дело серьёзное.

### Шаг 4. Общий вывод

Мы проанализировали данные по клиентам кредитной организации.
Была проведена предобработка данных, определены и исправлены ошибки выгрузки. Данные были приведены к правильным типам и нормированы. 
Также была проведена категоризация данных, для удобства анализа, и составлены словари категорий.

В результате были полученны таблицы, демонстрирующие связь между различными параметрами и возвратом кредита в срок.
По полученным данным были сформулированы выводы:
#### Зависимость от наличия детей
Было установлено, что процент должников с детьми выше, чем у должников без детей. При этом явной зависимости от количества детей выявлено не было.

#### Зависимость от семейного положения
Была выявлена зависимость процента должников от семейного положения. У заёмщиков без юридических обязательств (не женатые, гражданский брак), процент возвратов по кредитам меньше.

#### Зависимость от уровня дохода
Наиболее низкие проценты должников были выявлены у заёмщиков с низким доходом(<50000) и очень высоким (>250000). 

#### Зависимость от целей кредита
Было установлено, что кредиты на недвижимость и свадьбу возвращаются с меньшими задолженностями, чем на автомобили и образование.

#### Общий вывод
Можно сделать вывод, что незамужним, у которых 4 детей и средний доход, кредит на авто лучше не давать:)